# *Se* multiclass classification




01/11/2020





---



This notebook is part of a set of research experiments on Spanish *se* constructions. This notebook contains the code for training and testing *se* multiclass classification models. The data to do so is a gold standard corpus composed of 2,140 sentences (1,713 training sentences; 427 testing sentences) containing the word *se*. The multiclass classification task proposed in this notebook has to do with assigning a single defining tag to every instance of *se* that appears in the cotpus, that is, predicting the specific and exclusing properties of *se* in each sentence. The tag set used to annotate the corpus is composed of four tags: *expl*, *se-mark*, *iobj* and *obj*. The notebook is structured as follows:
    
1.   Preliminaries    
    1.1. Data loading    
    1.2. Data preparation

2.   Modelling and evaluation    
    2.1.   Baseline models    
    2.2.   Bag of words models     
    2.3.   HashingVectorizer models    
    2.4.   TF-IDF models    
    2.5.   BETO models    




---



## 1. Preliminaries

### 1.1. Data loading

Load train and test data from a `data` folder. Train and test partitions are set apart beforehand to use the same test dataset in case the train dataset grows. Besides, create a `preds` folder to save the predicitions the models generate. Since the tag distribution in the gold standard corpus is very unbalanced, four different scenarios strategies are tested to raise the number of correct cases of the less frequent categories: 
1.   Benchmark: using train and test datasets.    
2.   Using `f1-macro` parameter over GridSearch.    
3.   Using `class_weighted='balanced'` parameter over LinearSVC.     
4.   Using an oversampled version of the train dataset (`se_classification_balanced_train`).    



In [ ]:
import pandas as pd
train = pd.read_csv('./se_classification_train.csv', delimiter='\t', index_col='id')
train_oversampling = pd.read_csv('./se_classification_balanced_train.csv', delimiter='\t', index_col='id')
test = pd.read_csv('./se_classification_test.csv', delimiter='\t', index_col='id')

Check the content in saved in the variables `train` and `test`.

In [ ]:
train.head()

,text,se_tag
id,,
1,"Finalmente, el aragonés se hizo con su tercera...",expl
2,Les recomienda que visiten la web (www.farmace...,expl
3,"Ahí se desfondó el Deportivo, igual que Guarda...",expl
4,Dio un golpe de timón para adjudicarse la prim...,iobj
5,"Sólo se dirigía a mí para pedirme cosas, que s...",expl


In [ ]:
train_oversampling.head()

,text,se_tag
id,,
1,"Finalmente, el aragonés se hizo con su tercera...",expl
2,Les recomienda que visiten la web (www.farmace...,expl
3,"Ahí se desfondó el Deportivo, igual que Guarda...",expl
4,Dio un golpe de timón para adjudicarse la prim...,iobj
5,"Sólo se dirigía a mí para pedirme cosas, que s...",expl


In [ ]:
test.head()

,text,se_tag
id,,
1713,Las calabazas son el elemento decorativo con e...,se-mark
1714,"Según el alcalde de la localidad, José Dorado ...",expl
1715,En diez días empezará en Barcelona el juego me...,expl
1716,Sin inmutarse consideró importante ver los ext...,expl
1717,"Llega con desventaja a la segunda fase, donde ...",obj


### 1.2. Data preparation

Divide the train and test datasets into:    
*   X takes the variable *text* 
*   Y takes the variable *se_tag*

Besides, assign a number (0 to 3) to each of the four values the *se_tag* field might adquire through the LabelEncoder.

   

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_train = train['text'].values
X_train_oversampling = train_oversampling['text'].values
X_test = test['text'].values

label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(train['se_tag'].values)
Y_train_oversampling = label_encoder.fit_transform(train_oversampling['se_tag'].values)
Y_test = label_encoder.transform(test['se_tag'].values)

Check whether the number of tags is right.



In [ ]:
set(Y_train)

{0, 1, 2, 3}

In [ ]:
set(Y_train_oversampling)

{0, 1, 2, 3}

In [ ]:
set(Y_test)

{0, 1, 2, 3}

### 1.3 Additional packages

The following packages are required to run and optimize this model

In [ ]:
pip install transformers==3.5.1 scikit-optimize==0.8.1 spacy==3.0.* fasttext==0.9.2

     |████████████████████████████████| 1.3MB 17.6MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 12.8MB 226kB/s 
     |████████████████████████████████| 71kB 12.7MB/s 
     |████████████████████████████████| 1.1MB 24.9MB/s 
     |████████████████████████████████| 890kB 51.6MB/s 
     |████████████████████████████████| 2.9MB 48.9MB/s 
     |████████████████████████████████| 9.2MB 60.7MB/s 
     |████████████████████████████████| 460kB 46.3MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
     |████████████████████████████████| 122kB 56.5MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3092045 sha256=d1f3a85e47be4729e01b37429953391475688659800454a9f092419a7d550a60
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
  Created wheel for sacremoses: filename=sacremoses-0.0

### 1.4 General imports

In [ ]:
import numpy as np
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

## 2. Modelling and evaluation

In this section, eight different models are trained with different configurations to maximize the number of correct answers. After the training procedure, models are avaluated using *macro avg F-score*. Some of the parameters that govern the training procedure are defined here:

In [ ]:
TUNING_ITERATIONS = 30
VECTORIZER_BINARY = [True, False]
VECTORIZER_N_GRAM = [(1,1), (1,2), (1,3), (1,4), (1,5), (2,2), (3,3), (3,5), (5,5), (5,7), (7,7), (7,9), (10,10)]
RF_ESTIMATORS = [10, 100, 1000]
RF_MAX_DEPTH = [3, 5, 10, 15, 20, 25, 30, None]
SVC_C = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]

### 2.1. Base line models

#### 2.1.1 Base line model

This model asigns the most frequent tag (se-mark, tag 3) to the whole test set and checks the number of correct answers.

In [ ]:
import numpy as np
from sklearn.metrics import f1_score


baseline_preds = np.full(Y_test.shape, 3)
np.save('preds/baseline_preds', baseline_preds)
baseline_preds

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, baseline_preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.49      1.00      0.66       210

    accuracy                           0.49       428
   macro avg       0.12      0.25      0.16       428
weighted avg       0.24      0.49      0.32       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2.2. Bag of Words models

#### 2.2.1. Non-linear CountVectorizer

##### 2.2.1.1. Non-linear CountVectorizer (benchmark)

In [ ]:
cvrfgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

cvrfgs_model = RandomizedSearchCV(cvrfgs_model, params, n_iter=TUNING_ITERATIONS, n_jobs = -1, cv=StratifiedKFold(), random_state=12345, verbose=2)

cvrfgs_model.fit(X_train, Y_train)

print(cvrfgs_model.best_params_)

{'vectorizer__ngram_range': (3, 5), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': 30}


In [ ]:
cvrfgs_preds = cvrfgs_model.predict(X_test)
np.save('preds/cvrfgs_preds', cvrfgs_preds)
print(classification_report(Y_test, cvrfgs_preds))

              precision    recall  f1-score   support

           0       0.55      0.73      0.63       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.68      0.65      0.67       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.56      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.1.2. Non-linear CountVectorizer (`f1-macro`)

In [ ]:
cvrfgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

cvrfgs_f1_macro_model = RandomizedSearchCV(cvrfgs_model, params, scoring='f1_macro', n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvrfgs_f1_macro_model.fit(X_train, Y_train)

print(cvrfgs_f1_macro_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.7min finished


{'vectorizer__ngram_range': (3, 5), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': 30}


In [ ]:
cvrfgs_f1_macro_preds = cvrfgs_f1_macro_model.predict(X_test)
np.save('preds/cvrfgs_f1_macro_preds', cvrfgs_f1_macro_preds)
print(classification_report(Y_test, cvrfgs_f1_macro_preds))

              precision    recall  f1-score   support

           0       0.55      0.73      0.63       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.68      0.65      0.67       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.56      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.1.3. Non-linear CountVectorizer (`class_weight='balanced'`)

In [ ]:
cvrfgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

cvrfgs_class_weight_model = RandomizedSearchCV(cvrfgs_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvrfgs_class_weight_model.fit(X_train, Y_train)

print(cvrfgs_class_weight_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.9min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': None}


In [ ]:
cvrfgs_class_weight_model_preds = cvrfgs_class_weight_model.predict(X_test)
np.save('preds/cvrfgs_class_weight_model_preds', cvrfgs_class_weight_model_preds)
print(classification_report(Y_test, cvrfgs_class_weight_model_preds))

              precision    recall  f1-score   support

           0       0.58      0.65      0.61       173
           1       1.00      0.03      0.06        31
           2       0.00      0.00      0.00        14
           3       0.66      0.74      0.70       210

    accuracy                           0.63       428
   macro avg       0.56      0.35      0.34       428
weighted avg       0.63      0.63      0.59       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.1.4. Non-linear CountVectorizer (oversampling)

In [ ]:
cvrfgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

cvrfgs_oversampling_model = RandomizedSearchCV(cvrfgs_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvrfgs_oversampling_model.fit(X_train_oversampling, Y_train_oversampling)

print(cvrfgs_oversampling_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.7min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': None}


In [ ]:
cvrfgs_oversampling_preds = cvrfgs_oversampling_model.predict(X_test)
np.save('preds/cvrfgs_oversampling_preds', cvrfgs_oversampling_preds)
print(classification_report(Y_test, cvrfgs_oversampling_preds))

              precision    recall  f1-score   support

           0       0.57      0.61      0.59       173
           1       0.50      0.03      0.06        31
           2       0.00      0.00      0.00        14
           3       0.65      0.75      0.70       210

    accuracy                           0.62       428
   macro avg       0.43      0.35      0.34       428
weighted avg       0.59      0.62      0.59       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 2.2.2. Linear CountVectorizer

##### 2.2.2.1. Linear CountVectorizer (benchmark)

In [ ]:
cvovrgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

cvovrgs_model = RandomizedSearchCV(cvovrgs_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvovrgs_model.fit(X_train, Y_train)

print(cvovrgs_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.5min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
cvovrgs_preds = cvovrgs_model.predict(X_test)
np.save('preds/cvovrgs_preds', cvovrgs_preds)
print(classification_report(Y_test, cvovrgs_preds))

              precision    recall  f1-score   support

           0       0.58      0.65      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.65      0.71      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.2.2. Linear CountVectorizer (`f1_macro`)

In [ ]:
cvovrgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

cvovrgs_f1_macro_model = RandomizedSearchCV(cvovrgs_model, params, scoring='f1_macro', n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvovrgs_f1_macro_model.fit(X_train, Y_train)

print(cvovrgs_f1_macro_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.5min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
cvovrgs_f1_macro_preds = cvovrgs_f1_macro_model.predict(X_test)
np.save('preds/cvovrgs_f1_macro_preds', cvovrgs_f1_macro_preds)
print(classification_report(Y_test, cvovrgs_f1_macro_preds))

              precision    recall  f1-score   support

           0       0.58      0.65      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.65      0.71      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.2.3. Linear CountVectorizer( `class_weight='balanced'`)

In [ ]:
cvovrgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced')), n_jobs=-1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

cvovrgs_class_weight_model = RandomizedSearchCV(cvovrgs_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvovrgs_class_weight_model.fit(X_train, Y_train)

print(cvovrgs_class_weight_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.8min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
cvovrgs_class_weight_preds = cvovrgs_class_weight_model.predict(X_test)
np.save('preds/cvovrgs_class_weight_preds', cvovrgs_class_weight_preds)
print(classification_report(Y_test, cvovrgs_class_weight_preds))

              precision    recall  f1-score   support

           0       0.57      0.64      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.64      0.71      0.67       210

    accuracy                           0.61       428
   macro avg       0.30      0.34      0.32       428
weighted avg       0.54      0.61      0.57       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.2.4. Linear CountVectorizer(`f1_macro`, `class_weight='balanced'`)

In [ ]:
cvovrgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced')), n_jobs=-1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

cvovrgs_class_f1_model = RandomizedSearchCV(cvovrgs_model, params, scoring='f1_macro', n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvovrgs_class_f1_model.fit(X_train, Y_train)

print(cvovrgs_class_f1_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.9min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
cvovrgs_class_f1_preds = cvovrgs_class_f1_model.predict(X_test)
np.save('preds/cvovrgs_class_f1_preds', cvovrgs_class_f1_preds)
print(classification_report(Y_test, cvovrgs_class_f1_preds))

              precision    recall  f1-score   support

           0       0.57      0.64      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.64      0.71      0.67       210

    accuracy                           0.61       428
   macro avg       0.30      0.34      0.32       428
weighted avg       0.54      0.61      0.57       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.2.2.5. Linear CountVectorizer (oversampling)

In [ ]:
cvrfgs_model = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

cvrfgs_oversampling_model = RandomizedSearchCV(cvrfgs_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

cvrfgs_oversampling_model.fit(X_train_oversampling, Y_train_oversampling)

print(cvrfgs_oversampling_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 23.0min finished


{'vectorizer__ngram_range': (5, 5), 'vectorizer__binary': False, 'classifier__estimator__base_estimator__C': 0.1}


In [ ]:
cvrfgs_oversampling_preds = cvrfgs_oversampling_model.predict(X_test)
np.save('preds/cvrfgs_oversampling_preds', cvrfgs_oversampling_preds)
print(classification_report(Y_test, cvrfgs_oversampling_preds))

              precision    recall  f1-score   support

           0       0.55      0.68      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.66      0.66      0.66       210

    accuracy                           0.60       428
   macro avg       0.30      0.33      0.32       428
weighted avg       0.54      0.60      0.57       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2.3. HashingVectorizer models

#### 2.3.1. Non-linear HashingVectorizer




##### 2.3.1.1. Non-linear HashingVectorizer (benchmark)

In [ ]:
hvrf_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

hvrfgs_model = RandomizedSearchCV(hvrf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvrfgs_model.fit(X_train, Y_train)
print(hvrfgs_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 42.8min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': None}


In [ ]:
hvrfgs_preds = hvrfgs_model.predict(X_test)
np.save('preds/hvrfgs_preds', hvrfgs_preds)
print(classification_report(Y_test, hvrfgs_preds))

              precision    recall  f1-score   support

           0       0.55      0.66      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.66      0.69      0.67       210

    accuracy                           0.61       428
   macro avg       0.30      0.34      0.32       428
weighted avg       0.55      0.61      0.57       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.1.2. Non-linear HashingVectorizer (`f1_macro`)

In [ ]:
hvrf_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

hvrfgs_f1_macro_model = RandomizedSearchCV(hvrf_model, params, n_jobs = -1, scoring='f1_macro', cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvrfgs_f1_macro_model.fit(X_train, Y_train)
print(hvrfgs_f1_macro_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 43.2min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': None}


In [ ]:
hvrfgs_f1_macro_preds = hvrfgs_f1_macro_model.predict(X_test)
np.save('preds/hvrfgs_f1_macro_preds', hvrfgs_f1_macro_preds)
print(classification_report(Y_test, hvrfgs_f1_macro_preds))

              precision    recall  f1-score   support

           0       0.55      0.66      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.66      0.69      0.67       210

    accuracy                           0.61       428
   macro avg       0.30      0.34      0.32       428
weighted avg       0.55      0.61      0.57       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.1.2. Non-linear HashingVectorizer (`class_weight=balanced`)

In [ ]:
hvrf_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1, class_weight='balanced'))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

hvrfgs_class_weight_model = RandomizedSearchCV(hvrf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvrfgs_class_weight_model.fit(X_train, Y_train)
print(hvrfgs_class_weight_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 53.6min finished


{'vectorizer__ngram_range': (3, 5), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': 30}


In [ ]:
hvrfgs_class_weight_model_preds = hvrfgs_class_weight_model.predict(X_test)
np.save('preds/hvrfgs_f1_macro_preds', hvrfgs_class_weight_model_preds)
print(classification_report(Y_test, hvrfgs_class_weight_model_preds))

              precision    recall  f1-score   support

           0       0.54      0.63      0.58       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.63      0.68      0.65       210

    accuracy                           0.59       428
   macro avg       0.29      0.33      0.31       428
weighted avg       0.53      0.59      0.55       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.1.3. Non-linear HashingVectorizer (oversampling)

In [ ]:
hvrf_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1))
    ]
)

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
    'classifier__max_depth' : RF_MAX_DEPTH,
}

hvrfgs_oversampling_model = RandomizedSearchCV(hvrf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvrfgs_oversampling_model.fit(X_train_oversampling, Y_train_oversampling)
print(hvrfgs_oversampling_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 62.5min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000, 'classifier__max_depth': None}


In [ ]:
hvrfgs_oversampling_preds = hvrfgs_oversampling_model.predict(X_test)
np.save('preds/hvrfgs_oversampling_preds', hvrfgs_oversampling_preds)
print(classification_report(Y_test, hvrfgs_oversampling_preds))

              precision    recall  f1-score   support

           0       0.60      0.60      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.64      0.78      0.70       210

    accuracy                           0.62       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.62      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 2.3.2. Linear HashingVectorizer

##### 2.3.2.1. Linear HashingVectorizer (benchmark)


In [ ]:
hvovr_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

hvovr_model = RandomizedSearchCV(hvovr_model, params, n_jobs=-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvovr_model.fit(X_train, Y_train)
print(hvovr_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  8.8min finished


{'vectorizer__ngram_range': (5, 5), 'vectorizer__binary': False, 'classifier__estimator__base_estimator__C': 0.1}


In [ ]:
hvovr_preds = hvovr_model.predict(X_test)
np.save('preds/hvovr_preds', hvovr_preds)
print(classification_report(Y_test, hvovr_preds))

              precision    recall  f1-score   support

           0       0.53      0.68      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.65      0.64      0.65       210

    accuracy                           0.59       428
   macro avg       0.30      0.33      0.31       428
weighted avg       0.53      0.59      0.56       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.2.2. Linear HashingVectorizer (`f1_macro`)


In [ ]:
hvovr_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

hvovr_f1_macro_model = RandomizedSearchCV(hvovr_model, params, n_jobs=-1, scoring='f1_macro', cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvovr_f1_macro_model.fit(X_train, Y_train)
print(hvovr_f1_macro_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.8min finished


{'vectorizer__ngram_range': (5, 5), 'vectorizer__binary': False, 'classifier__estimator__base_estimator__C': 0.1}


In [ ]:
hvovr_f1_macro_preds = hvovr_f1_macro_model.predict(X_test)
np.save('preds/hvovr_f1_macro_preds', hvovr_f1_macro_preds)
print(classification_report(Y_test, hvovr_f1_macro_preds))

              precision    recall  f1-score   support

           0       0.53      0.68      0.60       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.65      0.64      0.65       210

    accuracy                           0.59       428
   macro avg       0.30      0.33      0.31       428
weighted avg       0.53      0.59      0.56       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.2.3. Linear HashingVectorizer (`class_weight`)

In [ ]:
hvovr_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced')), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

hvovr_class_weight_model = RandomizedSearchCV(hvovr_model, params, n_jobs=-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvovr_class_weight_model.fit(X_train, Y_train)
print(hvovr_class_weight_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  7.1min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
hvovr_class_weight_preds = hvovr_class_weight_model.predict(X_test)
np.save('preds/hvovr_class_weight_preds', hvovr_class_weight_preds)
print(classification_report(Y_test, hvovr_class_weight_preds))

              precision    recall  f1-score   support

           0       0.56      0.66      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.66      0.70      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.2.4. Linear HashingVectorizer (`f1_macro`, `class_weight`)

In [ ]:
hvovr_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced')), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

hvovr_class_f1_model = RandomizedSearchCV(hvovr_model, params, n_jobs=-1, scoring='f1_macro', cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvovr_class_f1_model.fit(X_train, Y_train)
print(hvovr_class_f1_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  7.1min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
hvovr_class_f1_preds = hvovr_class_f1_model.predict(X_test)
np.save('preds/hvovr_class_f1_preds', hvovr_class_f1_preds)
print(classification_report(Y_test, hvovr_class_f1_preds))

              precision    recall  f1-score   support

           0       0.56      0.66      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.66      0.70      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.3.2.5. Linear HashingVectorizer (oversampling)

In [ ]:
hvovr_model = Pipeline([
    ('vectorizer', HashingVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

hvovr_oversampling_model = RandomizedSearchCV(hvovr_model, params, n_jobs=-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

hvovr_oversampling_model.fit(X_train_oversampling, Y_train_oversampling)
print(hvovr_oversampling_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 27.1min finished


{'vectorizer__ngram_range': (1, 5), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 10.0}


In [ ]:
hvovr_oversampling_preds = hvovr_oversampling_model.predict(X_test)
np.save('preds/hvovr_oversampling_preds', hvovr_oversampling_preds)
print(classification_report(Y_test, hvovr_oversampling_preds))

              precision    recall  f1-score   support

           0       0.55      0.69      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.68      0.68      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2.4. TF-IDF models

#### 2.4.1 Non-linear TFD-IDF

##### 2.4.1.1. Non-linear TFI-DF (benchmark)

In [ ]:
tf_idf_rf_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
}

tf_idf_rf_gs_model = RandomizedSearchCV(tf_idf_rf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tf_idf_rf_gs_model.fit(X_train, Y_train)

print(tf_idf_rf_gs_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 10.6min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000}


In [ ]:
tf_idf_rf_gs_preds = tf_idf_rf_gs_model.predict(X_test)
np.save('preds/tf_idf_rf_gs_preds', tf_idf_rf_gs_preds)
print(classification_report(Y_test, tf_idf_rf_gs_preds))

              precision    recall  f1-score   support

           0       0.55      0.67      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.67      0.70      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.1.2. Non-linear TF-IDF (`f1_macro`)


In [ ]:
tfidfrf_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
}

tfidfrfgs_f1_macro_model = RandomizedSearchCV(tfidfrf_model, params, scoring='f1_macro', n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfrfgs_f1_macro_model.fit(X_train, Y_train)

print(tfidfrfgs_f1_macro_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 10.5min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000}


In [ ]:
tfidfrfgs_f1_macro_preds = tfidfrfgs_f1_macro_model.predict(X_test)
np.save('preds/tfidfrfgs_f1_macro_preds', tfidfrfgs_f1_macro_preds)
print(classification_report(Y_test, tfidfrfgs_f1_macro_preds))

              precision    recall  f1-score   support

           0       0.55      0.67      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.67      0.70      0.68       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.55      0.61      0.58       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.1.3. Non-linear TF-IDF (`class_weight='balanced'`)

In [ ]:
tf_idf_rf_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1, class_weight='balanced'))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
}

tf_idf_rf_gs_model = RandomizedSearchCV(tf_idf_rf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tf_idf_rf_gs_model.fit(X_train, Y_train)

print(tf_idf_rf_gs_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 11.1min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000}


In [ ]:
tf_idf_rf_gs_preds = tf_idf_rf_gs_model.predict(X_test)
np.save('preds/tf_idf_rf_gs_preds', tf_idf_rf_gs_preds)
print(classification_report(Y_test, tf_idf_rf_gs_preds))

              precision    recall  f1-score   support

           0       0.58      0.67      0.62       173
           1       1.00      0.03      0.06        31
           2       0.00      0.00      0.00        14
           3       0.67      0.72      0.70       210

    accuracy                           0.63       428
   macro avg       0.56      0.36      0.35       428
weighted avg       0.64      0.63      0.60       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.1.4. Non-linear TF-IDF (oversampling)

In [ ]:
tfidfrf_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs = -1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__n_estimators' : RF_ESTIMATORS,
}

tfidfrfgs_oversampling_model = RandomizedSearchCV(tfidfrf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfrfgs_oversampling_model.fit(X_train_oversampling, Y_train_oversampling)

print(tfidfrfgs_oversampling_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 16.9min finished


{'vectorizer__ngram_range': (5, 7), 'vectorizer__binary': True, 'classifier__n_estimators': 1000}


In [ ]:
tfidfrfgs_oversampling_preds = tfidfrfgs_oversampling_model.predict(X_test)
np.save('preds/tfidfrfgs_oversampling_preds', tfidfrfgs_oversampling_preds)
print(classification_report(Y_test, tfidfrfgs_oversampling_preds))

              precision    recall  f1-score   support

           0       0.59      0.60      0.59       173
           1       0.50      0.03      0.06        31
           2       0.00      0.00      0.00        14
           3       0.64      0.77      0.70       210

    accuracy                           0.62       428
   macro avg       0.43      0.35      0.34       428
weighted avg       0.59      0.62      0.59       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 2.4.2. Linear TF-IDF

##### 2.4.2.1. Linear TF-IDF (benchmark)

In [ ]:
tfidfovrsvc_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

tfidfovrsvcgs_model = RandomizedSearchCV(tfidfovrsvc_model, params, n_jobs =-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfovrsvcgs_model.fit(X_train, Y_train)

print(tfidfovrsvcgs_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   53.2s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  2.9min finished


{'vectorizer__ngram_range': (1, 5), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.001}


In [ ]:
tfidfovrsvcgs_preds = tfidfovrsvcgs_model.predict(X_test)
np.save('preds/tfidfovrsvcgs_preds', tfidfovrsvcgs_preds)
print(classification_report(Y_test, tfidfovrsvcgs_preds))

              precision    recall  f1-score   support

           0       0.57      0.75      0.65       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.72      0.70      0.71       210

    accuracy                           0.64       428
   macro avg       0.32      0.36      0.34       428
weighted avg       0.58      0.64      0.61       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.2.2. Linear TF-IDF (`f1_macro`)

In [ ]:
tfidfovrsvc_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

tfidfovrsvcgs_f1_macro_model = RandomizedSearchCV(tfidfovrsvc_model, params, scoring='f1_macro', n_jobs =-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfovrsvcgs_f1_macro_model.fit(X_train, Y_train)

print(tfidfovrsvcgs_f1_macro_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   52.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  2.9min finished


{'vectorizer__ngram_range': (1, 5), 'vectorizer__binary': True, 'classifier__estimator__base_estimator__C': 0.001}


In [ ]:
tfidfovrsvcgs_f1_macro_preds = tfidfovrsvcgs_f1_macro_model.predict(X_test)
np.save('preds/tfidfovrsvcgs_f1_macro_preds', tfidfovrsvcgs_f1_macro_preds)
print(classification_report(Y_test, tfidfovrsvcgs_f1_macro_preds))

              precision    recall  f1-score   support

           0       0.57      0.75      0.65       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.72      0.70      0.71       210

    accuracy                           0.64       428
   macro avg       0.32      0.36      0.34       428
weighted avg       0.58      0.64      0.61       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.2.3. Linear TF-IDF (`class_weight`)

In [ ]:
tfidfovrsvc_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced')), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

tfidfovrsvcgs_class_weight_model = RandomizedSearchCV(tfidfovrsvc_model, params, n_jobs =-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfovrsvcgs_class_weight_model.fit(X_train, Y_train)

print(tfidfovrsvcgs_class_weight_model.best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  2.9min finished


{'vectorizer__ngram_range': (3, 5), 'vectorizer__binary': False, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
tfidfovrsvcgs_class_weight_preds = tfidfovrsvcgs_class_weight_model.predict(X_test)
np.save('preds/tfidfovrsvcgs_class_weight_preds', tfidfovrsvcgs_class_weight_preds)
print(classification_report(Y_test, tfidfovrsvcgs_class_weight_preds))

              precision    recall  f1-score   support

           0       0.55      0.71      0.62       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.69      0.68      0.69       210

    accuracy                           0.62       428
   macro avg       0.31      0.35      0.33       428
weighted avg       0.56      0.62      0.59       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.2.4. Linear TF-IDF (`f1_macro`, `class_weight`)

In [ ]:
tfidfovrsvc_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(class_weight='balanced')), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

tfidfovrsvcgs_f1_class_model = RandomizedSearchCV(tfidfovrsvc_model, params, scoring='f1_macro', n_jobs =-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfovrsvcgs_f1_class_model.fit(X_train, Y_train)

print(tfidfovrsvcgs_f1_class_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.0min finished


{'vectorizer__ngram_range': (3, 5), 'vectorizer__binary': False, 'classifier__estimator__base_estimator__C': 0.0001}


In [ ]:
tfidfovrsvcgs_f1_class_preds = tfidfovrsvcgs_f1_class_model.predict(X_test)
np.save('preds/tfidfovrsvcgs_f1_class_preds', tfidfovrsvcgs_f1_class_preds)
print(classification_report(Y_test, tfidfovrsvcgs_f1_class_preds))

              precision    recall  f1-score   support

           0       0.55      0.71      0.62       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.69      0.68      0.69       210

    accuracy                           0.62       428
   macro avg       0.31      0.35      0.33       428
weighted avg       0.56      0.62      0.59       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### 2.4.2.5. Linear TF-IDF (oversampling)

In [ ]:
tfidfovrsvc_model = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char_wb')),
    ('classifier', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC()), n_jobs=-1))
])

params = {
    'vectorizer__binary' : VECTORIZER_BINARY,
    'vectorizer__ngram_range': VECTORIZER_N_GRAM,
    'classifier__estimator__base_estimator__C': SVC_C,
}

tfidfovrsvcgs_oversampling_model = RandomizedSearchCV(tfidfovrsvc_model, params, n_jobs =-1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

tfidfovrsvcgs_oversampling_model.fit(X_train_oversampling, Y_train_oversampling)

print(tfidfovrsvcgs_oversampling_model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 16.3min finished


{'vectorizer__ngram_range': (5, 5), 'vectorizer__binary': False, 'classifier__estimator__base_estimator__C': 0.1}


In [ ]:
tfidfovrsvcgs_oversampling_preds = tfidfovrsvcgs_oversampling_model.predict(X_test)
np.save('preds/tfidfovrsvcgs_oversampling_preds', tfidfovrsvcgs_oversampling_preds)
print(classification_report(Y_test, tfidfovrsvcgs_oversampling_preds))

              precision    recall  f1-score   support

           0       0.57      0.65      0.61       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.66      0.71      0.69       210

    accuracy                           0.61       428
   macro avg       0.31      0.34      0.32       428
weighted avg       0.56      0.61      0.58       428



### 2.5. Pre-trained language models (transformers)

#### 2.5.1. Collator

Collator function for batching the data for the model



In [ ]:
import torch

class TextClassificationCollator:
    """Data collator for a text classification problem"""
    
    def __init__(self, tokenizer):
        """Initializes the collator with a tokenizer"""
        self.tokenizer = tokenizer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def encode_texts(self, texts):
        """Transforms an iterable of texts into a dictionary of model input tensors, stored in the GPU"""
        # Tokenize and encode texts as tensors, with maximum length
        tensors = self.tokenizer.batch_encode_plus(texts, padding="longest", return_tensors="pt")
        # Move tensors to GPU
        for key in tensors:
            tensors[key] = tensors[key].to(self.device)
        return tensors
    
    def __call__(self, patterns):
        """Collate a batch of patterns
        
        Arguments:
            - patterns: iterable of tuples in the form (text, class)
            
        Output: dictionary of torch tensors ready for model input
        """
        # Check kind of input
        if len(patterns) < 1: raise ValueError(f"At least one pattern is required for training, found {len(patterns)}")
        if not isinstance(patterns[0], (tuple, str)): raise ValueError(f"Each pattern must be one text, or a tuple with text and label. Found {patterns[0]}")
        targets_provided = len(patterns[0]) == 2
        # Split texts and classes from the input list of tuples
        if targets_provided:
            train_idx, targets = zip(*patterns)
        else:
            train_idx = patterns
        # Encode inputs
        input_tensors = self.encode_texts(train_idx)
        if targets_provided:
          # Transform class labels to a tensor in GPU
          Y = torch.tensor(targets).long().to(self.device)
        # Return batch as a dictionary wikth all the inputs tensors and the labels
        batch = {**input_tensors}
        if targets_provided:
          batch["labels"] = Y
        return batch

#### 2.5.2. Model

Define a scikit-learn compatible class for a Tranformers model

In [ ]:
from copy import copy
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

class TransformersClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, pretrained_model='dccuchile/bert-base-spanish-wwm-uncased', 
                 learning_rate=5e-5, num_train_epochs=1, per_device_train_batch_size=8, per_device_eval_batch_size=128, 
                 attention_probs_dropout_prob=0.1, hidden_dropout_prob=0.1, output_dir="./transformers_model", ):
        self.pretrained_model = pretrained_model
        self.learning_rate = learning_rate
        self.num_train_epochs = num_train_epochs
        self.per_device_train_batch_size = per_device_train_batch_size 
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.hidden_dropout_prob = hidden_dropout_prob
        self.output_dir = output_dir
        self.per_device_eval_batch_size = per_device_eval_batch_size

    def fit(self, X, y):
        # Clear GPU memory
        torch.cuda.empty_cache()
        # Prepare config
        num_labels = len(set(y))
        config = AutoConfig.from_pretrained(self.pretrained_model, num_labels=num_labels)
        config.attention_probs_dropout_prob = self.attention_probs_dropout_prob
        config.hidden_dropout_prob = self.hidden_dropout_prob
        # Prepare tokenizer
        tokenizer = AutoTokenizer.from_pretrained(self.pretrained_model)
        # Build collator
        collator = TextClassificationCollator(tokenizer)
        # Initialize model
        model = AutoModelForSequenceClassification.from_pretrained(self.pretrained_model, config=config)
        # Prepare training args
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            overwrite_output_dir=True,
            per_device_eval_batch_size=self.per_device_eval_batch_size,
            disable_tqdm=True,
            learning_rate = self.learning_rate,
            num_train_epochs = self.num_train_epochs,
            per_device_train_batch_size = self.per_device_train_batch_size
        )
        # Initialize trainer
        self._trainer = Trainer(
            model=model,
            data_collator=collator,
            args=training_args,
            train_dataset=list(zip(X, y))
        )
        # Train
        self._trainer.train()

    def predict(self, X):
        preds = self._trainer.predict(X)
        return np.argmax(preds.predictions, axis=1)

    def score(self, X, y):
        preds = self.predict(X)
        return f1_score(y, preds, average="macro")

#### 2.5.3. Model training

Train basic model

In [ ]:
model = TransformersClassifier(pretrained_model='dccuchile/bert-base-spanish-wwm-uncased', num_train_epochs=4)
model.fit(X_train, Y_train)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncas

{'loss': 0.6036767578125, 'learning_rate': 2.0794392523364487e-05, 'epoch': 2.336448598130841}
{'epoch': 4.0}


Basic model metrics

In [ ]:
from sklearn.metrics import classification_report
preds = model.predict(X_test)
print(classification_report(Y_test, preds))  
# 0.66-0.67 with BETO uncased and 4 epochs

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       173
           1       0.69      0.65      0.67        31
           2       0.40      0.29      0.33        14
           3       0.93      0.87      0.90       210

    accuracy                           0.84       428
   macro avg       0.70      0.67      0.68       428
weighted avg       0.84      0.84      0.84       428



Run hyperparameters optimization

In [ ]:
%%time
from skopt import BayesSearchCV
from skopt.space.space import Integer, Real
from sklearn.model_selection import StratifiedKFold

param_grid = {
    "pretrained_model": ['dccuchile/bert-base-spanish-wwm-uncased', 'dccuchile/bert-base-spanish-wwm-cased'],
    "learning_rate": Real(1e-6, 1e-4, "log-uniform"),
    "num_train_epochs": Integer(1, 10, "uniform"),
    "per_device_train_batch_size": [4, 8, 16, 32, 64],
    "attention_probs_dropout_prob": Real(0, 0.9, "uniform"),
    "hidden_dropout_prob": Real(0, 0.9, "uniform"),
}

cv_method = StratifiedKFold()

metamodel = BayesSearchCV(model, param_grid, n_iter=30, verbose=3, cv=cv_method, random_state=12345, error_score=0.0)
metamodel.fit(X_train, Y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] attention_probs_dropout_prob=0.36712704740889385, hidden_dropout_prob=0.8900230698958589, learning_rate=1.552111989984059e-05, num_train_epochs=5, per_device_train_batch_size=32, pretrained_model=dccuchile/bert-base-spanish-wwm-uncased 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

{'epoch': 5.0}
[CV]  attention_probs_dropout_prob=0.36712704740889385, hidden_dropout_prob=0.8900230698958589, learning_rate=1.552111989984059e-05, num_train_epochs=5, per_device_train_batch_size=32, pretrained_model=dccuchile/bert-base-spanish-wwm-uncased, score=0.203, total= 1.6min
[CV] attention_probs_dropout_prob=0.36712704740889385, hidden_dropout_prob=0.8900230698958589, learning_rate=1.552111989984059e-05, num_train_epochs=5, per_device_train_batch_size=32, pretrained_model=dccuchile/bert-base-spanish-wwm-uncased 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s
Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were no

{'epoch': 5.0}
[CV]  attention_probs_dropout_prob=0.36712704740889385, hidden_dropout_prob=0.8900230698958589, learning_rate=1.552111989984059e-05, num_train_epochs=5, per_device_train_batch_size=32, pretrained_model=dccuchile/bert-base-spanish-wwm-uncased, score=0.208, total= 1.6min
[CV] attention_probs_dropout_prob=0.36712704740889385, hidden_dropout_prob=0.8900230698958589, learning_rate=1.552111989984059e-05, num_train_epochs=5, per_device_train_batch_size=32, pretrained_model=dccuchile/bert-base-spanish-wwm-uncased 


...

{'epoch': 10.0}
[CV]  attention_probs_dropout_prob=0.6093773596951091, hidden_dropout_prob=0.027175797976312522, learning_rate=2.007004297760599e-05, num_train_epochs=10, per_device_train_batch_size=64, pretrained_model=dccuchile/bert-base-spanish-wwm-cased, score=0.409, total= 3.0min
[CV] attention_probs_dropout_prob=0.6093773596951091, hidden_dropout_prob=0.027175797976312522, learning_rate=2.007004297760599e-05, num_train_epochs=10, per_device_train_batch_size=64, pretrained_model=dccuchile/bert-base-spanish-wwm-cased 


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

{'epoch': 10.0}
[CV]  attention_probs_dropout_prob=0.6093773596951091, hidden_dropout_prob=0.027175797976312522, learning_rate=2.007004297760599e-05, num_train_epochs=10, per_device_train_batch_size=64, pretrained_model=dccuchile/bert-base-spanish-wwm-cased, score=0.410, total= 3.0min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 15.1min finished
Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

{'loss': 0.874583251953125, 'learning_rate': 3.9386595113926067e-05, 'epoch': 1.1682242990654206}
{'loss': 0.6801448974609375, 'learning_rate': 3.4176728035364416e-05, 'epoch': 2.336448598130841}
{'loss': 0.5423587646484375, 'learning_rate': 2.8966860956802765e-05, 'epoch': 3.5046728971962615}
{'loss': 0.387064208984375, 'learning_rate': 2.3756993878241114e-05, 'epoch': 4.672897196261682}
{'loss': 0.230538818359375, 'learning_rate': 1.8547126799679467e-05, 'epoch': 5.841121495327103}
{'loss': 0.13231640625, 'learning_rate': 1.3337259721117818e-05, 'epoch': 7.009345794392523}
{'loss': 0.064142333984375, 'learning_rate': 8.12739264255617e-06, 'epoch': 8.177570093457945}
{'loss': 0.05944580078125, 'learning_rate': 2.9175255639945227e-06, 'epoch': 9.345794392523365}
{'epoch': 10.0}
CPU times: user 5h 35min 27s, sys: 1h 38min 25s, total: 7h 13min 53s
Wall time: 7h 24min 31s


In [ ]:
metamodel.best_params_

OrderedDict([('attention_probs_dropout_prob', 0.5609426282257046),
             ('hidden_dropout_prob', 0.0),
             ('learning_rate', 4.459646219248771e-05),
             ('num_train_epochs', 10),
             ('per_device_train_batch_size', 4),
             ('pretrained_model', 'dccuchile/bert-base-spanish-wwm-cased')])

Optimized model metrics

In [ ]:
from sklearn.metrics import classification_report
preds = metamodel.predict(X_test)
print(classification_report(Y_test, preds))

              precision    recall  f1-score   support

           0       0.78      0.84      0.81       173
           1       0.75      0.58      0.65        31
           2       0.29      0.29      0.29        14
           3       0.90      0.87      0.89       210

    accuracy                           0.82       428
   macro avg       0.68      0.65      0.66       428
weighted avg       0.82      0.82      0.82       428



Best optimized result

    OrderedDict([('attention_probs_dropout_prob', 0.371179371670504),
             ('hidden_dropout_prob', 0.0),
             ('learning_rate', 8.835158172392285e-05),
             ('num_train_epochs', 6),
             ('per_device_train_batch_size', 32),
             ('pretrained_model', 'dccuchile/bert-base-spanish-wwm-cased')])

                precision    recall  f1-score   support

              0       0.75      0.89      0.82       173
              1       0.71      0.65      0.68        31
              2       0.50      0.36      0.42        14
              3       0.95      0.84      0.89       210

        accuracy                           0.83       428
      macro avg       0.73      0.68      0.70       428
    weighted avg       0.84      0.83      0.83       428



### 2.6. Basic embeddings model

For reference we also provide experimental results with a simple model making use of pre-trained word embeddings. This model represents a text as the average of the word embeddings of every word in the text.

Install spacy model

In [ ]:
!python -m spacy download en_core_web_lg

2021-02-07 10:20:21.230153: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
     |████████████████████████████████| 778.8MB 23kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


Transform texts to embedding vectors

In [ ]:
import numpy as np
import spacy

def vectorize_texts(texts, spacy_model='en_core_web_lg'):
    nlp = spacy.load(spacy_model)
    return np.array([vectorize_text(text, nlp) for text in texts])

def vectorize_text(text, nlp):
    tokens = nlp(text)
    vector = np.zeros(tokens[0].vector.shape)
    n_vectors = 0
    for token in tokens:
        if token.has_vector:
            vector += token.vector
            n_vectors +=1
    return vector / n_vectors

In [ ]:
E_train = vectorize_texts(X_train)

In [ ]:
E_train_oversampling = vectorize_texts(X_train_oversampling)

In [ ]:
E_test = vectorize_texts(X_test)

#### 2.6.1. SpacyEmbeddings + RandomForestClassifier

In [ ]:
spacy_rf_model = RandomForestClassifier(random_state=42, n_jobs = -1)

params = {
    'n_estimators' : RF_ESTIMATORS,
}

spacy_rf_gs_model = RandomizedSearchCV(spacy_rf_model, params, n_jobs = -1, cv=StratifiedKFold(), n_iter=TUNING_ITERATIONS, random_state=12345, verbose=2)

spacy_rf_gs_model.fit(E_train, Y_train)

print(spacy_rf_gs_model.best_params_)  

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=30. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.0min finished


{'n_estimators': 1000}


In [ ]:
spacy_rf_gs_preds = spacy_rf_gs_model.predict(E_test)
np.save('preds/spacy_rf_gs_preds', spacy_rf_gs_preds)
print(classification_report(Y_test, spacy_rf_gs_preds))

              precision    recall  f1-score   support

           0       0.51      0.59      0.55       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.59      0.64      0.62       210

    accuracy                           0.55       428
   macro avg       0.28      0.31      0.29       428
weighted avg       0.50      0.55      0.52       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Very poor results for this modelling strategy

### 2.7. Word embeddings + recurrent mixing model

We further explore the previous results by devoloping a more advanced embeddings-based model: mixing word embeddings through a Gated Recurrent Unit layer.

First, load fasttext embeddings

In [ ]:
import fasttext.util

fasttext.util.download_model('es', if_exists='ignore')
ft = fasttext.load_model('cc.es.300.bin')

The following code defines a model with fasttext embeddings followed by a GRU mixing layer.

In [ ]:
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional, Dense, GlobalAveragePooling1D, GRU, Input, SpatialDropout1D, Dropout
from keras.preprocessing.sequence import pad_sequences 
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score

class FastTextGRUClassifier(BaseEstimator, ClassifierMixin):
    """Model implementing FastText embeddings for words followed by a GRU recurrent deep network"""
    def __init__(self, spatial_dropout=0.2, gru_layers=1, gru_units=80, gru_dropout=0.0, dense_layers=1, dense_units=16, dense_dropout=0.0,
                 max_tokenizer_words=15000, max_sequence=45, epochs=1, batch_size=64):
        self.spatial_dropout = spatial_dropout
        self.gru_layers = gru_layers
        self.gru_units = gru_units
        self.gru_dropout = gru_dropout
        self.dense_layers = dense_layers
        self.dense_units = dense_units
        self.dense_dropout = dense_dropout
        self.max_tokenizer_words = max_tokenizer_words
        self.max_sequence = max_sequence
        self.epochs = epochs
        self.batch_size = batch_size

    def _create_embedding_matrix(self, fasttext_model):
        """Creates a weight matrix for an Embedding layer using a fasttext_model and a Tokenizer"""
        
        # Compute mean and standard deviation for words embeddings
        all_embs = np.stack([
            fasttext_model.get_word_vector(word) for word in self._tokenizer.word_index.keys()
        ])

        emb_mean, emb_std = all_embs.mean(), all_embs.std()
        
        embedding_size = fasttext_model.get_dimension()
        
        embedding_matrix = np.random.normal(emb_mean, emb_std, (self._tokenizer.num_words, embedding_size))
        for word, i in self._tokenizer.word_index.items():
            if i >= self._tokenizer.num_words: break
            embedding_vector = fasttext_model.get_word_vector(word)
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                
        return embedding_matrix

    def fit(self, X, y, fasttext_model):
        # Prepare tokenizer
        self._tokenizer = Tokenizer(num_words=self.max_tokenizer_words)
        self._tokenizer.fit_on_texts(X)

        # Transform texts to sequences
        tokenized = self._tokenizer.texts_to_sequences(X)
        sequences = pad_sequences(tokenized, maxlen=self.max_sequence)

        # One-hot encode output labels
        y_hot = to_categorical(y)

        # Build embeddings matrix
        embedding_matrix = self._create_embedding_matrix(fasttext_model)

        # Build neural network
        inp = Input(shape=(self.max_sequence, ))
        x = Embedding(self.max_tokenizer_words, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False)(inp)
        x = SpatialDropout1D(self.spatial_dropout)(x)
        x = Bidirectional(GRU(self.gru_units, dropout=self.gru_dropout, return_sequences=True))(x)
        for _ in range(1, self.gru_layers):
            x = GRU(self.gru_units, dropout=self.gru_dropout, return_sequences=True)(x)
        x = GlobalAveragePooling1D()(x)
        for _ in range(self.dense_layers):
            x = Dense(self.dense_units, activation="relu")(x)
            x = Dropout(self.dense_dropout)(x)
        x = Dense(len(set(y)), activation="softmax")(x)
        model = Model(inputs=inp, outputs=x)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

        model.fit(sequences, y_hot, batch_size=self.batch_size, epochs=self.epochs, verbose=0)
        self._model = model

    def predict(self, X):
        # Transform texts to sequences
        tokenized = self._tokenizer.texts_to_sequences(X)
        sequences = pad_sequences(tokenized, maxlen=self.max_sequence)

        # Predict with model
        preds = self._model.predict(sequences)
        return np.argmax(preds, axis=1)

    def score(self, X, y):
        preds = self.predict(X)
        return f1_score(y, preds, average="macro")

Basic GRU (default parameters)

In [ ]:
gru_model = FastTextGRUClassifier(epochs=20)
gru_model.fit(X_train, Y_train, ft)

In [ ]:
gru_preds = gru_model.predict(X_test)
np.save('preds/gru_preds', gru_preds)
print(classification_report(Y_test, gru_preds))

              precision    recall  f1-score   support

           0       0.59      0.68      0.63       173
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        14
           3       0.70      0.76      0.73       210

    accuracy                           0.65       428
   macro avg       0.32      0.36      0.34       428
weighted avg       0.58      0.65      0.61       428



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Hyperoptimization

In [ ]:
%%time
from skopt import BayesSearchCV
from skopt.space.space import Integer, Real
from sklearn.model_selection import StratifiedKFold

param_grid = {
    "spatial_dropout": Real(0, 0.9, "uniform"),
    "gru_layers": [1, 2, 3],
    "gru_units": [16, 32, 64, 128, 256, 512, 1024],
    "gru_dropout": Real(0, 0.9, "uniform"),
    "dense_layers": [1, 2, 3],
    "dense_units" : [16, 32, 64, 128, 256, 512, 1024],
    "dense_dropout": Real(0, 0.9, "uniform"),
    "epochs": Integer(50, 200, "uniform"),
}

cv_method = StratifiedKFold()

gru_bs_model = BayesSearchCV(FastTextGRUClassifier(), param_grid, n_iter=30, verbose=3, cv=cv_method, random_state=12345, error_score=0.0, fit_params={"fasttext_model": ft})
gru_bs_model.fit(X_train, Y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] dense_dropout=0.36712704740889385, dense_layers=3, dense_units=256, epochs=109, gru_dropout=0.7118832782946124, gru_layers=3, gru_units=64, spatial_dropout=0.5377325391546796 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  dense_dropout=0.36712704740889385, dense_layers=3, dense_units=256, epochs=109, gru_dropout=0.7118832782946124, gru_layers=3, gru_units=64, spatial_dropout=0.5377325391546796, score=0.286, total=  37.2s
[CV] dense_dropout=0.36712704740889385, dense_layers=3, dense_units=256, epochs=109, gru_dropout=0.7118832782946124, gru_layers=3, gru_units=64, spatial_dropout=0.5377325391546796 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.2s remaining:    0.0s


[CV]  dense_dropout=0.36712704740889385, dense_layers=3, dense_units=256, epochs=109, gru_dropout=0.7118832782946124, gru_layers=3, gru_units=64, spatial_dropout=0.5377325391546796, score=0.316, total=  37.3s
[CV] dense_dropout=0.36712704740889385, dense_layers=3, dense_units=256, epochs=109, gru_dropout=0.7118832782946124, gru_layers=3, gru_units=64, spatial_dropout=0.5377325391546796 


...

[CV]  dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724, score=0.310, total= 2.3min
[CV] dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.6min remaining:    0.0s


[CV]  dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724, score=0.289, total= 2.3min
[CV] dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724 
[CV]  dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724, score=0.374, total= 2.3min
[CV] dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724 
[CV]  dense_dropout=0.0, dense_layers=1, dense_units=1024, epochs=110, gru_dropout=0.0, gru_layers=1, gru_units=1024, spatial_dropout=0.41227104827233724, score=0.338, total= 2.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.4min finished


CPU times: user 2h 18min 28s, sys: 20min 29s, total: 2h 38min 57s
Wall time: 6h 22min 40s


In [ ]:
gru_bs_model.best_params_

OrderedDict([('dense_dropout', 0.8390663304905417),
             ('dense_layers', 1),
             ('dense_units', 1024),
             ('epochs', 200),
             ('gru_dropout', 0.0),
             ('gru_layers', 2),
             ('gru_units', 1024),
             ('spatial_dropout', 0.0)])

In [ ]:
gru_bs_preds = gru_bs_model.predict(X_test)
np.save('preds/gru_bs_preds', gru_bs_preds)
print(classification_report(Y_test, gru_bs_preds))

              precision    recall  f1-score   support

           0       0.56      0.71      0.63       173
           1       0.29      0.19      0.23        31
           2       0.50      0.07      0.12        14
           3       0.71      0.64      0.68       210

    accuracy                           0.62       428
   macro avg       0.52      0.40      0.41       428
weighted avg       0.62      0.62      0.61       428



Best params:

      OrderedDict([('dense_dropout', 0.8390663304905417),
                  ('dense_layers', 1),
                  ('dense_units', 1024),
                  ('epochs', 200),
                  ('gru_dropout', 0.0),
                  ('gru_layers', 2),
                  ('gru_units', 1024),
                  ('spatial_dropout', 0.0)])

Best results:

                    precision    recall  f1-score   support

                0       0.56      0.71      0.63       173
                1       0.29      0.19      0.23        31
                2       0.50      0.07      0.12        14
                3       0.71      0.64      0.68       210

          accuracy                           0.62       428
        macro avg       0.52      0.40      0.41       428
      weighted avg       0.62      0.62      0.61       428
